In [5]:
import botocore
import dotenv
import os
import boto3


In [7]:
import boto3
import botocore # erros de boto3 para exceptions (404, ver se existe)
import os
import pandas as pd
import dask.dataframe as dd # utilizado para leitura de arquivos massivos

# ====================== #
#        Config          #
# ====================== # 

regiao='us-east-1'
nome_bucket_raw='raw-analisevoos-grupo2'
nome_bucket_trusted=''

client = boto3.client('s3', region_name=regiao)
s3 = boto3.resource('s3', region_name=regiao)

# ====================== #
#       Functions        #
# ====================== # 

def deletar_arquivo_se_existe(bucketName, filePath):
    try:  
        s3.Object(bucketName, filePath).load()
    except botocore.exceptions.ClientError as erro:
        # Arquivo não existe
        if erro.response['Error']['Code'] == '404':
            return 'arquivo não existe'
        else: 
        # Erro inesperado
            raise erro
    else:
        # Deletando arquivo (mais performático, eu acho)
        s3.Object(bucketName, filePath).delete()
        return 'arquivo deletado'
    
def subir_arquivo_deletando_se_existe(bucketName, filePath, fileNameOnBucket):
    deletar_arquivo_se_existe(bucketName, filePath)
    client.upload_file(filePath ,bucketName, fileNameOnBucket)



def limpar_arquivos_locais(nome):
    if os.path.exists(nome):
        os.remove(nome)

In [8]:
def subir_arquivo(bucketName, filePath, fileNameOnBucket):
    try:
        client.upload_file(filePath, bucketName, fileNameOnBucket)
        print(f"Arquivo {fileNameOnBucket} enviado para o bucket {bucketName} com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar o arquivo {fileNameOnBucket}: {e}")

In [10]:
subir_arquivo(nome_bucket_raw, "dados/raw/Dados Atividade/dadosconsumidor2024.csv", "dadosconsumidor2024.csv")

Erro ao enviar o arquivo dadosconsumidor2024.csv: Failed to upload dados/raw/Dados Atividade/dadosconsumidor2024.csv to raw-analisevoos-grupo2/dadosconsumidor2024.csv: An error occurred (ExpiredToken) when calling the CreateMultipartUpload operation: The provided token has expired.


In [41]:
import boto3
from botocore.exceptions import ClientError

dotenv.load_dotenv(dotenv.find_dotenv())


key_id = os.getenv("aws_access_key_id")
key_acess = os.getenv("aws_secret_access_key")
aws_token = os.getenv("aws_session_token")



s3 = boto3.client(
    's3',
    aws_access_key_id=key_id,
    aws_secret_access_key=key_acess,
    aws_session_token=aws_token
)


bucket_name = "raw-analise-voos-grupo2"   # nome único do bucket

# Criar o bucket
try:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': 'us-east-1'}  # se não for us-east-1
    )
    print(f"Bucket '{bucket_name}' criado com sucesso!")
except ClientError as e:
    print(f"Erro ao criar o bucket: {e}")

# Habilitar criptografia SSE-S3 automaticamente para novos objetos
try:
    s3.put_bucket_encryption(
        Bucket=bucket_name,
        ServerSideEncryptionConfiguration={
            'Rules': [
                {
                    'ApplyServerSideEncryptionByDefault': {
                        'SSEAlgorithm': 'AES256'  # SSE-S3
                    }
                }
            ]
        }
    )
    print("Criptografia SSE-S3 ativada no bucket.")
except ClientError as e:
    print(f"Erro ao ativar criptografia: {e}")

Erro ao criar o bucket: An error occurred (ExpiredToken) when calling the CreateBucket operation: The provided token has expired.
Erro ao ativar criptografia: An error occurred (ExpiredToken) when calling the PutBucketEncryption operation: The provided token has expired.


In [45]:
dotenv.load_dotenv(dotenv.find_dotenv())


key_id = os.getenv("aws_access_key_id")
key_acess = os.getenv("aws_secret_access_key")
aws_token = os.getenv("aws_session_token")



s3 = boto3.client(
    's3',
    aws_access_key_id=key_id,
    aws_secret_access_key=key_acess,
    aws_session_token=aws_token
)



bucket_name = 'raw-analisevoos-grupo2'
pasta_local = 'dados/raw/Dados Atividade/'

In [46]:
def upload_s3():
    for arquivo in os.listdir(pasta_local):
        caminho_arquivo = os.path.join(pasta_local, arquivo)
        
        try:
            s3.upload_file(caminho_arquivo, bucket_name, arquivo)
            print(f"Arquivo {arquivo} enviado para o bucket {bucket_name} com sucesso!")
            
            os.remove(caminho_arquivo)
            print(f"Arquivo {arquivo} apagado da pasta local.")
        
        except Exception as e:
            print(f"Erro ao enviar ou apagar o arquivo {arquivo}: {e}")
            
upload_s3()

Erro ao enviar ou apagar o arquivo aerodromospublicos.xls: Failed to upload dados/raw/Dados Atividade/aerodromospublicos.xls to raw-analisevoos-grupo2/aerodromospublicos.xls: An error occurred (ExpiredToken) when calling the PutObject operation: The provided token has expired.
Erro ao enviar ou apagar o arquivo aerodromospublicos.xlsx: Failed to upload dados/raw/Dados Atividade/aerodromospublicos.xlsx to raw-analisevoos-grupo2/aerodromospublicos.xlsx: An error occurred (ExpiredToken) when calling the PutObject operation: The provided token has expired.
Erro ao enviar ou apagar o arquivo dadosconsumidor2024.csv: Failed to upload dados/raw/Dados Atividade/dadosconsumidor2024.csv to raw-analisevoos-grupo2/dadosconsumidor2024.csv: An error occurred (ExpiredToken) when calling the CreateMultipartUpload operation: The provided token has expired.
Erro ao enviar ou apagar o arquivo Empresas Aereas.xlsx: Failed to upload dados/raw/Dados Atividade/Empresas Aereas.xlsx to raw-analisevoos-grupo2/E